<a href="https://colab.research.google.com/github/polevev/kaggle/blob/main/Stars_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('train_star.csv')
df.sample(5)

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

Обработка поля e_Plx и Plx


In [ ]:
df['TargetClass'] = df['TargetClass'].apply(lambda x: 1 if x == 'Giant' else 0)

In [ ]:
df.describe()

In [ ]:
import seaborn as sns

sns.heatmap(df.corr(numeric_only=True))

In [ ]:
data = df['TargetClass'].value_counts()

data

In [ ]:
df_origin = df.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

Scaler =  StandardScaler()

In [ ]:
df_scale = df[['Vmag','Plx', 'e_Plx', 'B-V', 'Amag']]

In [ ]:
df[['Vmag','Plx', 'e_Plx', 'B-V', 'Amag']] = Scaler.fit_transform(df_scale)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df['SpType'].value_counts()

In [ ]:
df_col = df['SpType'].value_counts()

In [ ]:
df_col[df_col>10]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onhot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

train_encoded = onhot_encoder.fit_transform(df[['SpType']])

In [ ]:
train_encoded

In [ ]:
df['SpType'].nunique()

In [ ]:
train_encoded_df = pd.DataFrame(train_encoded, columns = onhot_encoder.get_feature_names_out(['SpType']))

In [ ]:
train_encoded_df

In [ ]:
data = pd.concat([df.drop(['SpType'], axis=1), train_encoded_df], axis=1)

In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split
X_origin = df_origin.drop(columns=['TargetClass'],axis=1)
X = data.drop(columns=['TargetClass'],axis=1)
y = data['TargetClass']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.3,stratify=y, random_state=42)

In [ ]:
print(X_train.shape,X_test.shape)

In [ ]:
X.isna().sum()

In [ ]:
X_origin

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Шаг 1. создание модели
lr = KNeighborsClassifier()

# Шаг 2. обучение модели
lr.fit(X_train, y_train)

# Шаг 3. Предсказание на тестовых данных
y_pred = lr.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import GridSearchCV

#Задаем возможные параметры логистической регрессии(Только для GridSearchCV)
params = {
    'C': [100, 10, 1, 0.1],
    'penalty': ['l2', 'l1'],
    'solver': ['liblinear', 'saga']
}
#Создаем модель
model = LogisticRegression(random_state=42)
#Создаем обьект GridSearchCV
gs = GridSearchCV(
    model,
    params,
    cv=5,
    verbose=10,
    scoring='f1',
    refit=True
)
#Обучаем
gs.fit(X_train,y_train)

best_model = gs.best_estimator_

print('Лучшие параметры: ', gs.best_params_)

Лучшие параметры Logregr:  {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}

In [ ]:
best_score_lr = gs.best_score_
best_score_lr

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
params = {
    'max_depth': [3, 5, 7, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
#Создаем модель
model_tree = DecisionTreeClassifier(random_state=42)
#Создаем обьект GridSearchCV
gs_tree = GridSearchCV(
    model_tree,
    params,
    cv=5,
    verbose=10,
    scoring='f1',
    refit=True
)
#Обучаем
gs_tree.fit(X_train,y_train)

best_model_tree = gs_tree.best_estimator_
best_score_tree = gs_tree.best_score_

print('Лучшие параметры: ', gs_tree.best_params_)
print('Лучший скор: ', gs_tree.best_score_)

Лучшие параметры:  {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
#Создаем модель
model_forest = RandomForestClassifier(random_state=42)
#Создаем обьект GridSearchCV
gs_forest = GridSearchCV(
    model_forest,
    params,
    cv=5,
    verbose=10,
    scoring='f1',
    refit=True
)
#Обучаем
gs_forest.fit(X_train,y_train)

best_model_forest = gs_forest.best_estimator_
best_score_forest = gs_forest.best_score_

print('Лучшие параметры: ', gs_forest.best_params_)
print('Лучший скор: ', gs_forest.best_score_)

Лучшие параметры:  {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}

In [ ]:
!pip install catboost

In [ ]:
import catboost

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
X_train_origin, X_test_origin, _, _ = train_test_split(X_origin.values, y.values,
                                                       test_size=0.3,
                                                       random_state=42)

In [ ]:
X_origin

In [ ]:
#Задаем возможные параметры CatBoost
n_estimators = [50, 100, 200, 300]
learning_rates = [0.01, 0.05, 0.1, 0.2]

best_f1 = 0
best_params = {}

for n_estimators in n_estimators:
  for learning_rate in learning_rates:

    model = catboost.CatBoostClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            verbose=10,
            cat_features = [4]
        )

    model.fit(X_train_origin, y_train)

    y_pred_proba = model.predict(X_test_origin)

    f1 = f1_score(y_test, y_pred_proba)

    if f1 > best_f1:
        best_f1 = f1
        best_params = {'n_estimators': n_estimators, 'learning_rate': learning_rate}

print(f"Лучшие параметры: {best_params}")
print(f"Лучший f1_score: {best_f1:.4f}")

Лучшие параметры: {'n_estimators': 100, 'learning_rate': 0.2}

Нейросеть

In [ ]:
import torch

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(1686, 128)
        self.act1 = nn.Sigmoid()

        self.fc2 = nn.Linear(128, 1)
        self.act2 = nn.Sigmoid()


    def forward(self, x):

        x = self.fc1(x)
        x = self.act1(x)

        x = self.fc2(x)
        x = self.act2(x)
        return x

In [ ]:
train_loader = torch.utils.data.DataLoader(list(zip(X_train_tensor, y_train_tensor)), batch_size=64, shuffle=True)

In [ ]:
import tqdm
from tqdm.auto import tqdm

def train_stochastic(model, loader, criterion, optimizer, num_epoch):
    # цикл по количеству эпох обучения
    for t in tqdm(range(num_epoch)):
        epoch_loss = []

        # цикл по батчам даталоадера
        for X_batch, y_batch in loader:
            # Вычислим предсказания нашей модели
            y_pred = model(X_batch)

            # Посчитаем значение функции потерь  на полученном предсказании
            loss = criterion(y_pred, y_batch)
            epoch_loss.append(loss.item())

            # Выполним подсчёт новых градиентов
            loss.backward()
            # Выполним шаг градиентного спуска
            optimizer.step()
            # Обнулим сохраненные у оптимизатора значения градиентов
            # перед следующим шагом обучения
            optimizer.zero_grad()

        print("curr_loss", np.mean(epoch_loss))

    return model

In [ ]:
model = Net()

# лосс-функция — бинарная кросс-энтропия (aka LogLoss)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

In [ ]:
y_train_tensor = y_train_tensor.unsqueeze(1)

In [ ]:
model = train_stochastic(model, train_loader, criterion, optimizer, 150)

In [ ]:
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [ ]:
with torch.no_grad():
    nn_prediction = model(torch.FloatTensor(X_test_tensor))
    nn_prediction = nn_prediction.tolist()

nn_prediction = np.array([int(x[0] > 0.5) for x in nn_prediction])

In [ ]:
print(f1_score(y_test, nn_prediction))

In [ ]:
test = pd.read_csv('test_star.csv')

In [ ]:
test_scale = test[['Vmag','Plx', 'e_Plx', 'B-V', 'Amag']]
test[['Vmag','Plx', 'e_Plx', 'B-V', 'Amag']] = Scaler.transform(test_scale)

In [ ]:
test.head()

In [ ]:
test_encoded = onhot_encoder.transform(test[['SpType']])

In [ ]:
test_encoded_df = pd.DataFrame(test_encoded, columns = onhot_encoder.get_feature_names_out(['SpType']))

In [ ]:
test_data = pd.concat([test.drop(['SpType'], axis=1), test_encoded_df], axis=1)

In [ ]:
test_data

In [ ]:
test_data.columns

In [ ]:
X.columns

In [ ]:
y_pred_log = best_model.predict(test_data)

In [ ]:
y_pred_tree = best_model_tree.predict(test_data)

In [ ]:
y_pred_forest = best_model_forest.predict(test_data)

In [ ]:
y_pred_cat = model.predict_proba(test)

In [ ]:
test_data = torch.from_numpy(test_data.values).float()

In [ ]:
with torch.no_grad():
    y_pred_NN = model(torch.FloatTensor(test_data))
    y_pred_NN = y_pred_NN.tolist()

y_pred_NN = np.array([int(x[0] > 0.5) for x in y_pred_NN])

In [ ]:
submission = pd.read_csv('sample_submition_stars.csv', index_col='index')
submission

In [ ]:
submission['TargetClass'] = y_pred_NN

In [ ]:
submission.to_csv("/content/NN_submission.csv")

In [ ]:
submission.value_counts()

In [ ]:
submission['TargetClass'] = submission['TargetClass'].apply(lambda x: 1 if x>0.5 else 0)

In [ ]:
submission

Улучшение модели